Единственная задача: обобщить написанный на семинаре код и сымитировать работу одного большого отдела ABBYY, в котором есть три маленьких подотдела с лингвистами, программистами и комплингом. То есть, что у нас должно быть реализовано:

- родительский класс "работник"
- базовые классы "лингвист", "программист" и "компьютерный лингвист"
- у всех методы work
- классы "босс_лингвист", "босс_программист" и "босс_кл", которые могут наследовать (с подмешиванием) от общего класса "босс"
- у боссов в атрибутах сидят их подчиненные
- босс подотдела получает квесты от менеджера главного отдела и принуждает сотрудников работать
- в главном отделе есть метод для выдачи квестов
- соответственно, используем как наследование, так и композицию с делегированием

In [62]:
class Employee:
  coef = 1

  def __init__(self, name, surname):
    self.name = name
    self.surname = surname
    self.bankaccount = 0
    self.salary = 350

  def work(self, hours):
    print('Working...')
    self.bankaccount += self.salary * self.coef * hours

class Linguist(Employee):
  coef = 0.8

  def __init__(self, name, surname, publications=None):
    super().__init__(name, surname)
    if publications is None:
      self.publications = []
    else:
      self.publications = publications

  def publish(self, name_of_publ):
    self.publications.append(name_of_publ)
    Employee.work(self, 24)
    print(f'{self.name} {self.surname} has published a paper {name_of_publ} in Voprosy Yazykoznaniya')

  def readlingpapers(self, lingpaper):
    print('Reading...')
    print(f'{self.name} {self.surname} has read {lingpaper}')

class Programmer(Employee):
  coef = 2

  def __init__(self, name, surname, projects=None):
    super().__init__(name, surname)
    if projects is None:
      self.projects = []
    else:
      self.projects = projects

  def code(self, project):
    print('Coding...')
    Employee.work(self, 8)
    self.projects.append(project)
    print(f'{self.name} {self.surname} has committed a {project}')

class ComputationalLinguist (Linguist, Programmer):
  coef = 1.5

  def __init__(self, name, surname, projects, publications):
    super().__init__(name, surname, projects)
    super().__init__(name, surname, publications)

  def publishcode(self, name_of_paper_with_code):
    print('Working...')
    Employee.work(self, 32)
    self.publications.append(name_of_paper_with_code)
    self.projects.append(name_of_paper_with_code)
    print(f'{self.name} {self.surname} has published {name_of_paper_with_code} in ACL Papers')

class BOSS(Employee):

  def __init__(self, name, surname, workers=None):
    super().__init__(name, surname)
    if workers is None:
      self.workers = []
    else:
      self.workers = workers

  def hire(self, worker):
    if worker not in self.workers:
      self.workers.append(worker)
    else:
      print(f'{worker.name} {worker.surname} already works for you')

  def fire(self, worker):
    if worker in self.workers:
      self.workers.remove(worker)
    else:
      print(f'{worker.name} {worker.surname} has never worked for you')

  def workEveryone(self, hour):
    for worker in self.workers:
      worker.work(hour)

class KLBoss(BOSS):
  coef = 2,5

  def __init__(self, name, surname, workers=None):
    super().__init__(name, surname, workers)

  def makepublishcode(self, name_of_paper_with_code):
    for worker in self.workers:
      worker.publishcode(name_of_paper_with_code)

class ProgBoss(BOSS):
  coef = 4

  def __init__(self, name, surname, workers=None):
    super().__init__(name, surname, workers)

  def makecode(self, project):
    for worker in self.workers:
      worker.code(project)

class LingvoBoss(BOSS):
  coef = 2,7

  def __init__(self, name, surname, workers=None):
    super().__init__(name, surname, workers)

  def makepublish(self, name_of_publ):
    for worker in self.workers:
      worker.publish(name_of_publ)

class MainBoss(BOSS):
  coef = 5

  def __init__(self, name, surname, workers=None):
    super().__init__(name, surname, workers)

  def workEveryone(self, hours):
    for boss in self.workers:
      for worker in boss.workers:
        worker.work(hours)

  def checkproductivity(self):
    for boss in self.workers:
      for worker in boss.workers:
        print(f'{worker.name} {worker.surname} has earned {worker.bankaccount} grivens')

In [63]:
lngv = Linguist('Test', 'Linguist')

In [64]:
lngv.publish('Deixis')

Working...
Test Linguist has published a paper Deixis in Voprosy Yazykoznaniya


In [65]:
lngv.bankaccount

6720.0

In [66]:
ceo = MainBoss('Vera', 'Podlesskaya')

In [67]:
lboss = LingvoBoss('Yakov', 'Testelets')

In [68]:
lboss.hire(lngv)

In [69]:
ceo.hire(lboss)

In [70]:
ceo.workEveryone(3)

Working...


In [61]:
ceo.checkproductivity()

Test Linguist has earned 7560.0 grivens
